#Step 1: HuggingFace Setup and Model Loading

In [2]:
%pip install huggingface_hub
%pip install -U bitsandbytes
%pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


In [3]:
from transformers import pipeline
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-zh-en")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, StoppingCriteria, StoppingCriteriaList
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en", dtype="auto", device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/807k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [5]:
#code to load tianyi's drive
#moving to repo
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive', force_remount=True)

# Repo info
MYDRIVE = "/content/drive/MyDrive"
REPO_NAME = "chineseproverbs"
REPO_PATH = os.path.join(MYDRIVE, REPO_NAME)

# Go to MyDrive
os.chdir(MYDRIVE)

# Clone if missing, else pull
if not os.path.exists(REPO_PATH):
    print("Cloning repo...")
    !git clone https://github.com/art3misxmoon/chineseproverbs.git
else:
    print("Repo exists, pulling latest updates...")
    os.chdir(REPO_PATH)
    !git pull

# Move to repo folder
os.chdir(REPO_PATH)
print("Current working directory:", os.getcwd())
!ls


Mounted at /content/drive
Repo exists, pulling latest updates...
hint: You have divergent branches and need to specify how to reconcile them.
hint: You can do so by running one of the following commands sometime before
hint: your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.
fatal: Need to specify how to reconcile divergent branches.
Current working directory: /content/drive/MyDrive/chineseproverbs
Getting_started.ipynb	  idiomkb_train.csv    opus_mt_zh_en_finetuned
idiomkb_cleaned_refs.csv  idiomkb_val.csv      raw_data
idiomkb_test.csv	  opus100_cleaned.csv  UN_cleaned.csv


#Step 2: Load the Dataset

Example output genrated by the model before fine tuning.

In [6]:
prefix = "画蛇添足"
inputs = tokenizer(prefix, return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    num_return_sequences=1,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    pad_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Draw snakes and fill them up.


In [7]:
!pip install transformers datasets
import pandas as pd

# Change to your paths
# train_df = pd.read_csv("/content/chineseproverbs/idiomkb_train.csv")
# val_df = pd.read_csv("/content/chineseproverbs/idiomkb_val.csv")
# test_df = pd.read_csv("/content/chineseproverbs/idiomkb_test.csv")

#tianyi's paths
train_df = pd.read_csv("idiomkb_train.csv")
val_df = pd.read_csv("idiomkb_val.csv")
test_df = pd.read_csv("idiomkb_test.csv")

print(train_df.head())

  chinese                                            english source  \
0    无所不晓   know everything, be knowledgeable in all aspects   JSON   
1    琼枝玉叶      beautiful and valuable decoration or ornament   JSON   
2    先斩后奏  taking impulsive or hasty actions without seek...   JSON   
3    南来北往  travelling back and forth between the north an...   JSON   
4    言听计从             obey someone's advice without question   JSON   

                                           refs_list  
0  ['know everything, be knowledgeable in all asp...  
1  ['beautiful and valuable decoration or ornament']  
2  ['taking impulsive or hasty actions without se...  
3  ['travelling back and forth between the north ...  
4         ["obey someone's advice without question"]  


In [26]:
test_df[['chinese', 'refs_list']].head(10)


,chinese,refs_list
0,艰苦创业,['start a business in difficult circumstances']
1,风雨同舟,['to weather the storm together']
2,斩草除根,"['eliminating the root cause', 'solving a prob..."
3,惊魂未定,['still in a state of shock and fear']
4,额手称庆,"['to pat oneself on the back', 'to be pleased ..."
5,傲然挺立,['standing proudly and confidently']
6,惊天动地,"['earth-shattering, extremely shocking']"
7,取信于民,['gaining trust and support from the people']
8,壁垒森严,"['having a strict and formidable defense', 'al..."
9,探囊取物,"['easily get what one wants, effortlessly obta..."


In [8]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df[['chinese', 'refs_list']])
val_dataset = Dataset.from_pandas(val_df[['chinese', 'refs_list']])
test_dataset = Dataset.from_pandas(test_df[['chinese', 'refs_list']])

#Step 3: Supervised Fine Tuning

In [9]:
%pip install transformers datasets sacrebleu
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model_name = "Helsinki-NLP/opus-mt-zh-en"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.5 MB/s eta 0:00:00


In [10]:
MAX_LEN = 256

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.bos_token is None:
    tokenizer.bos_token = tokenizer.eos_token
if tokenizer.eos_token is None:
    tokenizer.eos_token = tokenizer.eos_token

def tokenize_translation_batch(batch):
    inputs = batch["chinese"]
    targets = batch["refs_list"]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LEN,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        model_targets = tokenizer(
            targets,
            max_length=MAX_LEN,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = model_targets["input_ids"]
    return model_inputs

train_tok = train_dataset.map(
    tokenize_translation_batch,
    batched=True,
    batch_size=1024
    )
val_tok = val_dataset.map(
    tokenize_translation_batch,
    batched=True,
    batch_size=1024
    )
test_tok = test_dataset.map(
    tokenize_translation_batch,
    batched=True,
    batch_size=1024
    )

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

Map:   0%|          | 0/6904 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/864 [00:00<?, ? examples/s]

Map:   0%|          | 0/864 [00:00<?, ? examples/s]

In [11]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./opus_mt_zh_en_finetuned",
    num_train_epochs=4,   #changed to 4
    per_device_train_batch_size=32, #changed to 32 for a100 gpu
    per_device_eval_batch_size=32, #changed to 32
    learning_rate=5e-5,   #changed, safer LR for small dataset
    predict_with_generate=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1, #changed from 0.03 to 0.1 to improve convergence

    #added
    generation_max_length=64,
    eval_strategy="epoch", #evaluate once per epoch
    save_strategy="epoch",

    # eval_steps=25,
    # save_steps=250,
    save_total_limit=2,
    logging_strategy="steps",
    logging_steps=50,

    optim="adamw_torch",
    fp16=True,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipython-input-1128195345.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [16]:
#tianyi logging into huggingface
!pip install huggingface_hub --upgrade


In [18]:
trainer.train()
trainer.save_model()
# #Save the finetuned model
# model.push_to_hub("username/opus_mt_zh_en_finetuned")
# tokenizer.push_to_hub("username/opus_mt_zh_en_finetuned")

#saving locally for tianyi
trainer.save_model("./opus_mt_zh_en_finetuned_idioms")
tokenizer.save_pretrained("./opus_mt_zh_en_finetuned_idioms")

# Zip for download
!zip -r opus_mt_zh_en_finetuned_idioms.zip ./opus_mt_zh_en_finetuned_idioms

from google.colab import files
files.download("opus_mt_zh_en_finetuned_idioms.zip")

Epoch,Training Loss,Validation Loss
1,0.088400,0.134051
2,0.078400,0.133521
3,0.068400,0.133723
4,0.061000,0.134496


  adding: opus_mt_zh_en_finetuned_idioms/ (stored 0%)
  adding: opus_mt_zh_en_finetuned_idioms/config.json (deflated 63%)
  adding: opus_mt_zh_en_finetuned_idioms/generation_config.json (deflated 43%)
  adding: opus_mt_zh_en_finetuned_idioms/model.safetensors (deflated 7%)
  adding: opus_mt_zh_en_finetuned_idioms/tokenizer_config.json (deflated 69%)
  adding: opus_mt_zh_en_finetuned_idioms/special_tokens_map.json (deflated 47%)
  adding: opus_mt_zh_en_finetuned_idioms/vocab.json (deflated 68%)
  adding: opus_mt_zh_en_finetuned_idioms/source.spm (deflated 46%)
  adding: opus_mt_zh_en_finetuned_idioms/target.spm (deflated 49%)
  adding: opus_mt_zh_en_finetuned_idioms/training_args.bin (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Step 4: Evaluation

In [ ]:
import sacrebleu

#For faster testing
#test_tok = test_tok.select(range(200))
preds = trainer.predict(test_tok)
decoded_preds = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)
references = [[r] for r in test_df["refs_list"].tolist()]

bleu = sacrebleu.corpus_bleu(decoded_preds, references)
print("BLEU score:", bleu.score)

In [31]:
#tianyi running eval

# ==========================
# Full BLEU + BERTScore Debug Function
# ==========================
import sacrebleu
import evaluate
import ast
from time import time

def parse_refs(ref):
    """
    Parse refs_list cell into a list of strings.
    Handles string representation of list, nested lists, or empty lists.
    """
    try:
        r = ast.literal_eval(ref)
        if isinstance(r, list):
            flat_r = []
            for x in r:
                if isinstance(x, list):
                    flat_r.extend(x)
                else:
                    flat_r.append(str(x))
            return [s for s in flat_r if s.strip()]
        else:
            return [str(r)]
    except:
        return [str(ref)]

def evaluate_mt_bleu_bertscore_debug(trainer, tokenizer, test_df, test_tok, n_print=5):
    """
    Evaluate model on test set with BLEU and BERTScore, print sample outputs.

    Args:
        trainer: HuggingFace Seq2SeqTrainer
        tokenizer: tokenizer
        test_df: pandas DataFrame with 'chinese' and 'refs_list' columns
        test_tok: tokenized Dataset
        n_print: number of sample translations to print

    Returns:
        dict with BLEU, BERTScore F1 mean, elapsed time
    """
    start_time = time()

    # Ensure refs_list is properly parsed
    test_df['refs_list'] = test_df['refs_list'].apply(parse_refs)

    # Generate translations
    print("Generating translations...")
    preds = trainer.predict(test_tok)  # No batch_size here
    decoded_preds = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)

    # Prepare references
    references = test_df['refs_list'].tolist()
    refs_for_sacrebleu = list(zip(*references))  # transpose for BLEU

    # --- BLEU ---
    bleu = sacrebleu.corpus_bleu(decoded_preds, refs_for_sacrebleu)

    # --- BERTScore ---
    print("Computing BERTScore...")
    bertscore_metric = evaluate.load("bertscore")
    bertscore_res = bertscore_metric.compute(
        predictions=decoded_preds,
        references=[r[0] if len(r) > 0 else "" for r in references],
        lang="en"
    )

    elapsed = time() - start_time

    # --- Print evaluation results ---
    print("\n===== Evaluation Results =====")
    print(f"BLEU: {bleu.score:.2f}")
    print(f"BERTScore F1 (mean): {sum(bertscore_res['f1'])/len(bertscore_res['f1']):.4f}")
    print(f"Elapsed time: {elapsed/60:.2f} minutes\n")

    # --- Print first n sample translations ---
    print("===== Sample Translations =====")
    for i in range(min(n_print, len(decoded_preds))):
        src = test_df['chinese'][i]
        ref = references[i][0] if references[i] else "EMPTY"
        pred = decoded_preds[i]
        print(f"Source: {src}")
        print(f"Reference: {ref}")
        print(f"Prediction: {pred}")
        print("---")

    return {
        "bleu": bleu.score,
        "bertscore_f1": sum(bertscore_res['f1'])/len(bertscore_res['f1']),
        "elapsed_sec": elapsed
    }


In [32]:
results = evaluate_mt_bleu_bertscore_debug(trainer, tokenizer, test_df, test_tok, n_print=10)


Generating translations...


Computing BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== Evaluation Results =====
BLEU: 2.28
BERTScore F1 (mean): 0.8471
Elapsed time: 0.34 minutes

===== Sample Translations =====
Source: 艰苦创业
Reference: start a business in difficult circumstances
Prediction: ['to work hard to start a business']
---
Source: 风雨同舟
Reference: to weather the storm together
Prediction: ['to work together in a harmonious and harmonious manner']
---
Source: 斩草除根
Reference: eliminating the root cause
Prediction: ['eliminate the root']
---
Source: 惊魂未定
Reference: still in a state of shock and fear
Prediction: ['being in a state of shock and undecided']
---
Source: 额手称庆
Reference: to pat oneself on the back
Prediction: ["to celebrate with one's hands"]
---
Source: 傲然挺立
Reference: standing proudly and confidently
Prediction: ['standing proud and firm']
---
Source: 惊天动地
Reference: earth-shattering, extremely shocking
Prediction: ['astonishing and dramatic']
---
Source: 取信于民
Reference: gaining trust and support from the people
Prediction: ['seeking credibility fr